In [1]:
# notebook to load the raw govt (CPCB/DPCC/IMD) data in the Excel sheet format and save them as csvs with latitude 
# and longitude fields

%matplotlib inline

from glob import glob
from openpyxl import Workbook
from openpyxl import load_workbook
import json
from datetime import datetime
from datetime import timedelta
import time
import os, sys
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# load all the Excel sheets from the source folder on Dropbox

# sheets = sorted(glob('data/govdata/*.xlsx'), key=lambda name: int(name.split('/data')[1].replace('.xlsx','')))
sheets = sorted(glob('/home/shivar/Dropbox/Delhi Pollution/07_Data_Secondary/CPCB_Oct2018/Raw/*.xlsx'))
for sh in sheets:
    print(sh)

/home/shivar/Dropbox/Delhi Pollution/07_Data_Secondary/CPCB_Oct2018/Raw/AnandVihar_DPCC.xlsx
/home/shivar/Dropbox/Delhi Pollution/07_Data_Secondary/CPCB_Oct2018/Raw/AshokVihar_DPCC.xlsx
/home/shivar/Dropbox/Delhi Pollution/07_Data_Secondary/CPCB_Oct2018/Raw/AyaNagar_IMD.xlsx
/home/shivar/Dropbox/Delhi Pollution/07_Data_Secondary/CPCB_Oct2018/Raw/BurariCrossing_IMD.xlsx
/home/shivar/Dropbox/Delhi Pollution/07_Data_Secondary/CPCB_Oct2018/Raw/CRRIMathuraRoad_IMD.xlsx
/home/shivar/Dropbox/Delhi Pollution/07_Data_Secondary/CPCB_Oct2018/Raw/DKSSR_DPCC.xlsx
/home/shivar/Dropbox/Delhi Pollution/07_Data_Secondary/CPCB_Oct2018/Raw/DTU_CPCB.xlsx
/home/shivar/Dropbox/Delhi Pollution/07_Data_Secondary/CPCB_Oct2018/Raw/DwarkaSector8_DPCC.xlsx
/home/shivar/Dropbox/Delhi Pollution/07_Data_Secondary/CPCB_Oct2018/Raw/EastArjunNagar_CPCB.xlsx
/home/shivar/Dropbox/Delhi Pollution/07_Data_Secondary/CPCB_Oct2018/Raw/IGIAirport_IMD.xlsx
/home/shivar/Dropbox/Delhi Pollution/07_Data_Secondary/CPCB_Oct2018/Raw/

In [3]:
sample = load_workbook(sheets[0])

In [4]:
sample.sheetnames

['Sheet1']

In [5]:
# define function to read just Excel file as an example, to
# understand the format, number of entries and so on
    
def myconv(strval):
    if strval=='None':
        return np.nan
    else:
        return float(strval)

# nrows = 24 * 60 // 15
# for rr in range(nrows):
data = []
rr = 0
while True:
    date = sample['Sheet1'].cell(14 + rr, 1).value
    if date is None: break
    value1 = myconv(sample['Sheet1'].cell(14 + rr, 3).value)
    value2 = myconv(sample['Sheet1'].cell(14 + rr, 4).value)
    
    # to re-format the date and time in the same format as the Kaiterra dts
    dobj = datetime.strptime(date, '%d-%m-%Y %H:%M')
    data.append([dobj.strftime('%Y-%m-%d %H:%M:%S') + '+05:30', value1, value2])
    rr += 1

In [6]:
print(len(data), data[0][0], data[-1][0])

23520 2018-03-01 00:00:00+05:30 2018-10-31 23:45:00+05:30


In [7]:
data

[['2018-03-01 00:00:00+05:30', 163.0, 525.0],
 ['2018-03-01 00:15:00+05:30', 163.0, 525.0],
 ['2018-03-01 00:30:00+05:30', 163.0, 525.0],
 ['2018-03-01 00:45:00+05:30', 163.0, 525.0],
 ['2018-03-01 01:00:00+05:30', 172.0, 703.0],
 ['2018-03-01 01:15:00+05:30', 172.0, 703.0],
 ['2018-03-01 01:30:00+05:30', 172.0, 703.0],
 ['2018-03-01 01:45:00+05:30', 172.0, 703.0],
 ['2018-03-01 02:00:00+05:30', 154.0, 746.0],
 ['2018-03-01 02:15:00+05:30', 154.0, 746.0],
 ['2018-03-01 02:30:00+05:30', 154.0, 746.0],
 ['2018-03-01 02:45:00+05:30', 154.0, 746.0],
 ['2018-03-01 03:00:00+05:30', 188.0, 685.0],
 ['2018-03-01 03:15:00+05:30', 188.0, 685.0],
 ['2018-03-01 03:30:00+05:30', 188.0, 685.0],
 ['2018-03-01 03:45:00+05:30', 188.0, 685.0],
 ['2018-03-01 04:00:00+05:30', 153.0, 516.0],
 ['2018-03-01 04:15:00+05:30', 153.0, 516.0],
 ['2018-03-01 04:30:00+05:30', 153.0, 516.0],
 ['2018-03-01 04:45:00+05:30', 153.0, 516.0],
 ['2018-03-01 05:00:00+05:30', 195.0, 707.0],
 ['2018-03-01 05:15:00+05:30', 195

In [8]:
# sample plot of the data as an example illustration

plt.figure(figsize=(14, 5))
plt.plot(np.asarray([ent[1] for ent in data]), label='PM2.5')
plt.plot(np.asarray([ent[2] for ent in data]), label='PM10')
plt.legend()
plt.show()
plt.close()

In [9]:
import json

# reading in the metadata to get the latitudes and longitudes
locsdict = dict()

with open('data/gov_metadata.json')as fl:
    stations = json.load(fl)

for station in stations['map']['station_list']:
    nm = station['station_name']
    locsdict[nm] = (float(station['latitude']), float(station['longitude']))

In [10]:
print(len(locsdict))
locsdict

134


{'Lodhi Road, New Delhi - IMD': (28.5918245, 77.2273074),
 'Sector - 125, Noida, UP - UPPCB': (28.5447608, 77.3231257),
 'CRRI Mathura Road, New Delhi - IMD': (28.5512005, 77.2735737),
 'Burari Crossing, New Delhi - IMD': (28.7256504, 77.2011573),
 'ITO, New Delhi - CPCB': (28.6316945, 77.2494387),
 'Pusa, New Delhi - IMD': (28.639645, 77.146263),
 'DTU, New Delhi - CPCB': (28.7500499, 77.1112615),
 'North Campus, DU, New Delhi - IMD': (28.6573814, 77.1585447),
 'Shadipur, New Delhi - CPCB': (28.6514781, 77.1473105),
 'Aya Nagar, New Delhi - IMD': (28.4706914, 77.1099364),
 'Vasundhara, Ghaziabad, UP - UPPCB': (28.6603346, 77.3572563),
 'NSIT Dwarka, New Delhi - CPCB': (28.60909, 77.0325413),
 'Sector - 62, Noida, UP - IMD': (28.6245479, 77.3577104),
 'IHBAS, Dilshad Garden,New Delhi - CPCB': (28.6811736, 77.3025234),
 'BWSSB Kadabesanahalli, Bengaluru - CPCB': (12.9352049, 77.6814488),
 'Vikas Sadan, Gurgaon, Haryana - HSPCB': (28.4501238, 77.0263051),
 'Police Commissionerate, Jaipur

In [11]:
# correct errors (see notes.txt in the same folder as this notebook)
# (1) remove DU north campus
# (2) swap RK Puram and Punjabi Bagh locations
locsdict.__delitem__('North Campus, DU, New Delhi - IMD')
locsdict['Punjabi Bagh, Delhi - DPCC'] = (28.674045, 77.131023)
locsdict['R K Puram, New Delhi - DPCC'] = (28.563262, 77.186937)

In [12]:
# now read data for all the monitors and export into dataframe format
import pandas as pd

def todf(sheet):

    data = []
    rr = 0
    
    locname = sheet.cell(5, 2).value
    latitude, longitude = locsdict[locname]
    
    while True:
        date = sheet.cell(14 + rr, 1).value
        if date is None: 
            break
        
        value1 = myconv(sheet.cell(14 + rr, 3).value) # pm2.5
        value2 = myconv(sheet.cell(14 + rr, 4).value) # pm10
        
        # to re-format the date and time in the same format as the Kaiterra dts
        dobj = datetime.strptime(date, '%d-%m-%Y %H:%M')
        data.append([dobj.strftime('%Y-%m-%d %H:%M:%S') + '+05:30', value1, value2])
        rr += 1
    
    dtindex = [ent[0] for ent in data]
    valarr = np.asarray([(ent[1], ent[2], latitude, longitude) for ent in data])
    df = pd.DataFrame(valarr, index=dtindex, columns=['pm25', 'pm10', 'latitude', 'longitude'])
    df['location'] = locname
    df.index.name = 'timestamp_round'
    
    return df

In [13]:
df = todf(load_workbook(sheets[0])['Sheet1'])

In [14]:
df

pm25   pm10   latitude  longitude  \
timestamp_round                                                 
2018-03-01 00:00:00+05:30  163.0  525.0  28.646835  77.316032   
2018-03-01 00:15:00+05:30  163.0  525.0  28.646835  77.316032   
2018-03-01 00:30:00+05:30  163.0  525.0  28.646835  77.316032   
2018-03-01 00:45:00+05:30  163.0  525.0  28.646835  77.316032   
2018-03-01 01:00:00+05:30  172.0  703.0  28.646835  77.316032   
2018-03-01 01:15:00+05:30  172.0  703.0  28.646835  77.316032   
2018-03-01 01:30:00+05:30  172.0  703.0  28.646835  77.316032   
2018-03-01 01:45:00+05:30  172.0  703.0  28.646835  77.316032   
2018-03-01 02:00:00+05:30  154.0  746.0  28.646835  77.316032   
2018-03-01 02:15:00+05:30  154.0  746.0  28.646835  77.316032   
2018-03-01 02:30:00+05:30  154.0  746.0  28.646835  77.316032   
2018-03-01 02:45:00+05:30  154.0  746.0  28.646835  77.316032   
2018-03-01 03:00:00+05:30  188.0  685.0  28.646835  77.316032   
2018-03-01 03:15:00+05:30  188.0  685.0  28.646835  77.316032   
2018-03-01 03:30:00+05:30  188.0  685.0  28.646835  77.316032   
2018-03-01 03:45:00+05:30  188.0  685.0  28.646835  77.316032   
2018-03-01 04:00:00+05:30  153.0  516.0  28.646835  77.316032   
2018-03-01 04:15:00+05:30  153.0  516.0  28.646835  77.316032   
2018-03-01 04:30:00+05:30  153.0  516.0  28.646835  77.316032   
2018-03-01 04:45:00+05:30  153.0  516.0  28.646835  77.316032   
2018-03-01 05:00:00+05:30  195.0  707.0  28.646835  77.316032   
2018-03-01 05:15:00+05:30  195.0  707.0  28.646835  77.316032   
2018-03-01 05:30:00+05:30  195.0  707.0  28.646835  77.316032   
2018-03-01 05:45:00+05:30  195.0  707.0  28.646835  77.316032   
2018-03-01 06:00:00+05:30  181.0  606.0  28.646835  77.316032   
2018-03-01 06:15:00+05:30  181.0  606.0  28.646835  77.316032   
2018-03-01 06:30:00+05:30  181.0  606.0  28.646835  77.316032   
2018-03-01 06:45:00+05:30  181.0  606.0  28.646835  77.316032   
2018-03-01 07:00:00+05:30  174.0  550.0  28.646835  77.316032   
2018-03-01 07:15:00+05:30  174.0  550.0  28.646835  77.316032   
...                          ...    ...        ...        ...   
2018-10-31 16:30:00+05:30  124.0  429.0  28.646835  77.316032   
2018-10-31 16:45:00+05:30  132.0  287.0  28.646835  77.316032   
2018-10-31 17:00:00+05:30  132.0  287.0  28.646835  77.316032   
2018-10-31 17:15:00+05:30  132.0  287.0  28.646835  77.316032   
2018-10-31 17:30:00+05:30  132.0  287.0  28.646835  77.316032   
2018-10-31 17:45:00+05:30  207.0  466.0  28.646835  77.316032   
2018-10-31 18:00:00+05:30  207.0  466.0  28.646835  77.316032   
2018-10-31 18:15:00+05:30  207.0  466.0  28.646835  77.316032   
2018-10-31 18:30:00+05:30  207.0  466.0  28.646835  77.316032   
2018-10-31 18:45:00+05:30  233.0  631.0  28.646835  77.316032   
2018-10-31 19:00:00+05:30  233.0  631.0  28.646835  77.316032   
2018-10-31 19:15:00+05:30  233.0  631.0  28.646835  77.316032   
2018-10-31 19:30:00+05:30  233.0  631.0  28.646835  77.316032   
2018-10-31 19:45:00+05:30  272.0  757.0  28.646835  77.316032   
2018-10-31 20:00:00+05:30  272.0  757.0  28.646835  77.316032   
2018-10-31 20:15:00+05:30  272.0  757.0  28.646835  77.316032   
2018-10-31 20:30:00+05:30  272.0  757.0  28.646835  77.316032   
2018-10-31 20:45:00+05:30  276.0  849.0  28.646835  77.316032   
2018-10-31 21:00:00+05:30  276.0  849.0  28.646835  77.316032   
2018-10-31 21:15:00+05:30  276.0  849.0  28.646835  77.316032   
2018-10-31 21:30:00+05:30  276.0  849.0  28.646835  77.316032   
2018-10-31 21:45:00+05:30  293.0  825.0  28.646835  77.316032   
2018-10-31 22:00:00+05:30  293.0  825.0  28.646835  77.316032   
2018-10-31 22:15:00+05:30  293.0  825.0  28.646835  77.316032   
2018-10-31 22:30:00+05:30  293.0  825.0  28.646835  77.316032   
2018-10-31 22:45:00+05:30  371.0    NaN  28.646835  77.316032   
2018-10-31 23:00:00+05:30  371.0    NaN  28.646835  77.316032   
2018-10-31 23:15:00+05:30  371.0    NaN  28.646835  77.316032   
2018-10-31 23:30:00+05:30  371.0    NaN  28.646835  77.316032  

In [15]:
# saving individual dataframes as individual csvs, and also saving a combined BIG csv

# also save the locations
locs_list = []

dfs_list = []
for fii, fpath in enumerate(sheets):
    print('{}/{}'.format(fii + 1, len(sheets)))
    if fpath.endswith('NorthCampusDU_IMD.xlsx'):
        continue
    sheet = load_workbook(fpath)
    df = todf(sheet['Sheet1'])
    savedir, savename = os.path.split(fpath)
    monitorid = savename[:-5]
    df['monitor_id'] = monitorid
    df.reset_index(inplace=True)
    df.set_index(['monitor_id', 'timestamp_round'], inplace=True)
    locs_list.append((monitorid, df.latitude[0], df.longitude[0], df.location[0]))
    df.drop(axis=1, labels=['latitude', 'longitude', 'location'], inplace=True)
    dfs_list.append(df)
    savename = 'govdata_15min_' + monitorid + '.csv'
    df.to_csv(os.path.join(savedir, savename))

pd.concat(dfs_list).to_csv(os.path.join(savedir, 'govdata_15min_panel.csv'))

1/34
2/34
3/34
4/34
5/34
6/34
7/34
8/34
9/34
10/34
11/34
12/34
13/34
14/34
15/34
16/34
17/34
18/34
19/34
20/34
21/34
22/34
23/34
24/34
25/34
26/34
27/34
28/34
29/34
30/34
31/34
32/34
33/34
34/34


In [16]:
savedir

'/home/shivar/Dropbox/Delhi Pollution/07_Data_Secondary/CPCB_Oct2018/Raw'

In [20]:
locs_list

[('AnandVihar_DPCC', 28.646835, 77.316032, 'Anand Vihar, Delhi - DPCC'),
 ('AshokVihar_DPCC', 28.695381, 77.181665, 'Ashok Vihar, Delhi - DPCC'),
 ('AyaNagar_IMD', 28.4706914, 77.1099364, 'Aya Nagar, New Delhi - IMD'),
 ('BurariCrossing_IMD',
  28.7256504,
  77.2011573,
  'Burari Crossing, New Delhi - IMD'),
 ('CRRIMathuraRoad_IMD',
  28.5512005,
  77.2735737,
  'CRRI Mathura Road, New Delhi - IMD'),
 ('DKSSR_DPCC',
  28.498571,
  77.26484,
  'Dr. Karni Singh Shooting Range, Delhi - DPCC'),
 ('DTU_CPCB', 28.7500499, 77.1112615, 'DTU, New Delhi - CPCB'),
 ('DwarkaSector8_DPCC',
  28.5710274,
  77.0719006,
  'Dwarka-Sector 8, Delhi - DPCC '),
 ('EastArjunNagar_CPCB',
  28.6556017,
  77.2859318,
  'East Arjun Nagar, Delhi - CPCB'),
 ('IGIAirport_IMD',
  28.5627763,
  77.1180053,
  'IGI Airport (T3), New Delhi - IMD'),
 ('IHBAS_CPCB',
  28.6811736,
  77.3025234,
  'IHBAS, Dilshad Garden,New Delhi - CPCB'),
 ('ITO_CPCB', 28.6316945, 77.2494387, 'ITO, New Delhi - CPCB'),
 ('JNS_DPCC', 28.580

In [18]:
# colnames = ['Monitor ID', 'Latitude', 'Longitude', 'Location']
# collen_0 = max([len(colnames[0])] + [len(tup[0]) for tup in locs_list]) + 1
# collen_1 = max([len(colnames[1])] + [len(str(tup[1])) for tup in locs_list]) + 1
# collen_2 = max([len(colnames[2])] + [len(str(tup[2])) for tup in locs_list]) + 1

# fout = open(os.path.join(savedir, 'govdata_locations.csv'), 'w')
# fout.write('{{:{}}},{{:>{}}},{{:>{}}}, {{}}\n'.format(collen_0, collen_1, collen_2).format(*colnames))

# for locid, lat, lon, locname in locs_list:
#     fout.write('{{:{}}},{{:>{}}},{{:>{}}}, {{}}\n'.format(collen_0, collen_1, collen_2).format(locid, str(lat), str(lon), '"' + locname + '"'))
# fout.close()

In [28]:
# create dataframe and export to csv (above does not work)
locs_df = pd.DataFrame.from_records(locs_list, columns=['Monitor ID', 'Latitude', 'Longitude', 'Location'])
locs_df.set_index('Monitor ID', inplace=True)
locs_df

Latitude  Longitude  \
Monitor ID                                    
AnandVihar_DPCC        28.646835  77.316032   
AshokVihar_DPCC        28.695381  77.181665   
AyaNagar_IMD           28.470691  77.109936   
BurariCrossing_IMD     28.725650  77.201157   
CRRIMathuraRoad_IMD    28.551201  77.273574   
DKSSR_DPCC             28.498571  77.264840   
DTU_CPCB               28.750050  77.111261   
DwarkaSector8_DPCC     28.571027  77.071901   
EastArjunNagar_CPCB    28.655602  77.285932   
IGIAirport_IMD         28.562776  77.118005   
IHBAS_CPCB             28.681174  77.302523   
ITO_CPCB               28.631694  77.249439   
JNS_DPCC               28.580280  77.233829   
Jahangirpuri_DPCC      28.732820  77.170633   
LodhiRoad_IMD          28.591825  77.227307   
MDCNS_DPCC             28.611281  77.237738   
MandirMarg_DPCC        28.636429  77.201067   
Mundaka_DPCC           28.684678  77.076574   
NSIT_CPCB              28.609090  77.032541   
NehruNagar_DPCC        28.567890  77.250515   
OkhlaPhase2_DPCC       28.530785  77.271255   
Patparganj_DPCC        28.623748  77.287205   
PunjabiBagh_DPCC       28.674045  77.131023   
Pusa_DPCC              28.639645  77.146262   
Pusa_IMD               28.639645  77.146263   
RKPuram_DPCC           28.563262  77.186937   
Rohini_DPCC            28.732528  77.119920   
Shadipur_CPCB          28.651478  77.147311   
Sirifort_CPCB          28.550425  77.215938   
SoniaVihar_DPCC        28.710508  77.249485   
SriAurobindoMarg_DPCC  28.531346  77.190156   
VivekVihar_DPCC        28.672342  77.315260   
Wazirpur_DPCC          28.699793  77.165453   

                                                               Location  
Monitor ID                                                               
AnandVihar_DPCC                               Anand Vihar, Delhi - DPCC  
AshokVihar_DPCC                               Ashok Vihar, Delhi - DPCC  
AyaNagar_IMD                                 Aya Nagar, New Delhi - IMD  
BurariCrossing_IMD                     Burari Crossing, New Delhi - IMD  
CRRIMathuraRoad_IMD                  CRRI Mathura Road, New Delhi - IMD  
DKSSR_DPCC                 Dr. Karni Singh Shooting Range, Delhi - DPCC  
DTU_CPCB                                          DTU, New Delhi - CPCB  
DwarkaSector8_DPCC                       Dwarka-Sector 8, Delhi - DPCC   
EastArjunNagar_CPCB                      East Arjun Nagar, Delhi - CPCB  
IGIAirport_IMD                        IGI Airport (T3), New Delhi - IMD  
IHBAS_CPCB                       IHBAS, Dilshad Garden,New Delhi - CPCB  
ITO_CPCB                                          ITO, New Delhi - CPCB  
JNS_DPCC                         Jawaharlal Nehru Stadium, Delhi - DPCC  
Jahangirpuri_DPCC                            Jahangirpuri, Delhi - DPCC  
LodhiRoad_IMD                               Lodhi Road, New Delhi - IMD  
MDCNS_DPCC             Major Dhyan Chand National Stadium, Delhi - DPCC  
MandirMarg_DPCC                           Mandir Marg, New Delhi - DPCC  
Mundaka_DPCC                                      Mundaka, Delhi - DPCC  
NSIT_CPCB                                 NSIT Dwarka, New Delhi - CPCB  
NehruNagar_DPCC                               Nehru Nagar, Delhi - DPCC  
OkhlaPhase2_DPCC                            Okhla Phase-2, Delhi - DPCC  
Patparganj_DPCC                                Patparganj, Delhi - DPCC  
PunjabiBagh_DPCC                             Punjabi Bagh, Delhi - DPCC  
Pusa_DPCC                                            Pusa, Delhi - DPCC  
Pusa_IMD                                          Pusa, New Delhi - IMD  
RKPuram_DPCC                                R K Puram, New Delhi - DPCC  
Rohini_DPCC                                        Rohini, Delhi - DPCC  
Shadipur_CPCB                                Shadipur, New Delhi - CPCB  
Sirifort_CPCB                                Sirifort, New Delhi - CPCB  
SoniaVihar_DPCC                               Sonia Vihar, Delhi - DPCC  
SriAurobin

In [29]:
locs_df.to_csv(os.path.join(savedir, 'govdata_locations.csv'))